In [1]:
## Split min-ind3 text file into two different text files for encoding and decoding ##
import tensorflow as tf
import numpy as np
from tensorflow import keras
!pip install keras_metrics
import keras_metrics
np.random.seed(1)
tf.random.set_seed(2)
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
%cd /content/
import tensorflow as tf
import numpy as np
from tensorflow import keras
!pip install keras_metrics
import keras_metrics
np.random.seed(1)
tf.random.set_seed(2)
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
%cd /content/
## split text file ##
with open('min-ind3.txt','r',encoding='utf-8') as f:
  lines = f.read().split('\n')
import sklearn
from sklearn.model_selection import train_test_split
train_val,test = train_test_split(lines,test_size=0.2) # here split size can be change with test_size parameter
with open('min-ind_test.txt','w',encoding='utf-8') as w:
  for line in test:
    w.write(line+'\n')

with open('min-ind_train.txt','w',encoding='utf-8') as w:
  for line in train_val:
    w.write(line+'\n')
!pip install sentencepiece
lines_1 = []
lines_2 = []
with open('min-ind_train.txt','r',encoding='utf-8') as min:
  for line in min.readlines():
    split = line.split(',')
    if split!=['\n']:
      lines_1.append(split[0])
      lines_2.append(split[1])
with open('min-ind_input.txt','w',encoding='utf-8') as enc_txt:
  for line in lines_1:
    enc_txt.write(line+'\n')

with open('min-ind_target.txt','w',encoding='utf-8') as tar_txt:
  for line in lines_2:
    tar_txt.write(line)

import sentencepiece as spm
model_inp = spm.SentencePieceTrainer.train("--input=min-ind_input.txt --model_prefix=input --vocab_size=30")
sp_inp = spm.SentencePieceProcessor()
sp_inp.load('input.model')
#import sentencepiece as spm
model_tar = spm.SentencePieceTrainer.train("--input=min-ind_target.txt --model_prefix=target --vocab_size=30")
sp_tar = spm.SentencePieceProcessor()
sp_tar.load('target.model')

### Prepare data ###
input_encode = []
target_encode = []
with open('min-ind_input.txt','r',encoding='utf-8') as inp:
  for line in inp:
    input_encode.append(sp_inp.encode_as_pieces(line))
with open('min-ind_target.txt','r',encoding='utf-8') as tar:
  for line in tar:
    target_encode.append(['\t']+sp_tar.encode_as_pieces(line)+['\n'])

## Create a dictionary for characters mapping ##
input_subwords = np.unique(np.array([tk for tok in input_encode for tk in tok]))
input_subwords = np.append(input_subwords," ")
target_subwords = np.unique(np.array([tk for tok in target_encode for tk in tok]))
target_subwords = np.append(target_subwords," ")
input_token_index = dict([(char,i) for i,char in enumerate(sorted(list(input_subwords)))])
target_token_index = dict([(char,i) for i,char in enumerate(sorted(list(target_subwords)))])

num_encoder_tokens= len(input_subwords)
num_decoder_tokens = len(target_subwords)

max_encoder_seq_length = max([len(words) for words in input_encode])
max_decoder_seq_length = max([len(words) for words in target_encode])

encoder_input_data = np.zeros((len(input_encode),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(target_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

for i,(enc_word,tar_word) in enumerate(zip(input_encode,target_encode)):
  for t,char in enumerate(enc_word):
    encoder_input_data[i,t,input_token_index[char]] = 1.0
  decoder_input_data[i,t+1 :,input_token_index[" "]] = 1.0 # encoding space at the end of encoder word for decoder input to reach to next word for encoding
  for t,char in enumerate(tar_word):
    decoder_input_data[i,t,target_token_index[char]] = 1.0
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]] = 1.0
  decoder_input_data[i,t+1 :,target_token_index[" "]] = 1.0
  decoder_target_data[i,t:,target_token_index[" "]] = 1.0


/content
/content
     |████████████████████████████████| 1.2MB 11.4MB/s 


In [3]:
## 5-fold BI-LSTM ##
latent_dim_1 = 128
latent_dim_2 = 256
latent_dim_3 = 512
kfold_acc = []
kf = KFold(n_splits=5,random_state=96, shuffle = True) # 5 folds for training and validation sets
save_dir = '/content/'
fold_var = 5
batch_size = 128
epochs = 10
latent_dim = 256 # edit LSTM units
num_samples = 14032

import matplotlib.pyplot as plt
%matplotlib inline
kf = KFold(10)
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(input_encode[:11222])),decoder_target_data)):
  print('K={}'.format(i+1))
  encoder_inputs = keras.Input(shape=(None,num_encoder_tokens))
  encoder_outputs,forward_h,forward_c,backward_h,backward_c= keras.layers.Bidirectional(keras.layers.LSTM(latent_dim_2, return_state=True),merge_mode='concat',name='encoder_lstm1')(encoder_inputs)
  #encoder_outputs = encoder(encoder_inputs)
  #print(encoder)
  #encoder_outputs = keras.layers.Bidirectional(keras.layers.LSTM(latent_dim_2),merge_mode='concat',name='encoder_lstm2')(encoder)
  #print(encoder_outputs)
  #encoder_inputs = embedding_layer_input(encoder_inputs)
  # We discard `encoder_outputs` and only keep the states.
  state_h = tf.keras.layers.Concatenate()([forward_h,forward_c])
  state_c = tf.keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [state_h,state_c] # hidden and cell state
  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  #decoder_inputs = embedding_layer_output(decoder_inputs)
  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, return_sequences=True, return_state=True)(decoder_inputs, initial_state=encoder_states)
  #decoder_outputs, _, _ ,*_= decoder_lstm(decoder_inputs, initial_state=encoder_states)


  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_outputs) 

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  encoder_input_data_t = pd.Series(list(encoder_input_data)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data)).iloc[val_index].to_numpy()


  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])
  x_val = [encoder_input_data_val,decoder_input_data_val]
  y_val = decoder_target_data_val
  model.compile(
    optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  results = model.fit(
      [encoder_input_data_t, decoder_input_data_t],
      decoder_target_data_t,
      batch_size=batch_size,
      epochs=epochs,
      validation_data =(x_val,y_val)
  )
  '''
  plt.plot(results.history['accuracy'])
  plt.plot(results.history['val_accuracy'])
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend(['train','val'],loc='upper right')
  plt.title('Model Accuracy')
  plt.show()
  '''
  model.save('model_bilstm_256_{}-Fold{}'.format(fold_var,i+1))
  model = keras.models.load_model("model_bilstm_256_{}-Fold{}".format(fold_var,i+1)) # Using 5-Fold cross validated model
  encoder_inputs = model.input[0]  # input_1
  encoder_outputs,state_h_enc,state_c_enc,backward_h,backward_c  = model.layers[1].output  # lstm_1
  states_h = keras.layers.Concatenate()([state_h_enc,state_c_enc])
  states_c = keras.layers.Concatenate()([backward_h,backward_c])
  #print(states_h)
  encoder_states = [states_h,states_c]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim_3,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim_3,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
  decoder_lstm = model.layers[5]
  decoder_outputs, state_h_dec,state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs #, 
  )
  decoder_states = [state_h_dec,state_c_dec]
  decoder_dense = model.layers[6]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )
  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)
      #print(states_value)
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens,h,c = decoder_model.predict([target_seq] + states_value)
          # Sample a token
         # print('output_tokens',output_tokens)
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char
          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True
          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0
          states_value = [h,c]
      return decoded_sentence

    ## Split min-ind3 text file into two different text files for encoding and decoding ##
  '''
  lines_1 = []
  lines_2 = []
  with open('min-ind_test.txt','r',encoding='utf-8') as min:
    for line in min.readlines():
      split = line.split(',')
      if split!=['\n']:
        lines_1.append(split[0])
        lines_2.append(split[1])
  with open('min-ind_input_test.txt','w',encoding='utf-8') as enc_txt:
    for line in lines_1:
      enc_txt.write(line+'\n')

  with open('min-ind_target_test.txt','w',encoding='utf-8') as tar_txt:
    for line in lines_2:
      tar_txt.write(line)

  model_inp = spm.SentencePieceTrainer.train("--input=min-ind_input_test.txt --model_prefix=input_test --vocab_size=30")
  sp_inp = spm.SentencePieceProcessor()
  sp_inp.load('input_test.model')
  #import sentencepiece as spm
  model_tar = spm.SentencePieceTrainer.train("--input=min-ind_target_test.txt --model_prefix=target_test --vocab_size=30")
  sp_tar = spm.SentencePieceProcessor()
  sp_tar.load('target_test.model')

  ### Prepare data ###
  input_encode = []
  target_encode = []
  with open('min-ind_input_test.txt','r',encoding='utf-8') as inp:
    for line in inp:
      input_encode.append(sp_inp.encode_as_pieces(line))
  with open('min-ind_target_test.txt','r',encoding='utf-8') as tar:
    for line in tar:
      target_encode.append(['\t']+sp_tar.encode_as_pieces(line)+['\n'])

  ## Create a dictionary for characters mapping ##
  
  input_subwords = np.unique(np.array([tk for tok in input_encode for tk in tok]))
  input_subwords = np.append(input_subwords," ")
  target_subwords = np.unique(np.array([tk for tok in target_encode for tk in tok]))
  target_subwords = np.append(target_subwords," ")
  input_token_index = dict([(char,i) for i,char in enumerate(sorted(list(input_subwords)))])
  target_token_index = dict([(char,i) for i,char in enumerate(sorted(list(target_subwords)))])
  
  num_encoder_tokens= len(input_subwords)
  num_decoder_tokens = len(target_subwords)

  max_encoder_seq_length = max([len(words) for words in input_encode])
  max_decoder_seq_length = max([len(words) for words in target_encode])

  encoder_input_data = np.zeros((len(input_encode),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
  decoder_input_data = np.zeros((len(input_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
  decoder_target_data = np.zeros((len(target_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
  
  
  for i,(enc_word,tar_word) in enumerate(zip(input_encode,target_encode)):
    for t,char in enumerate(enc_word):
      encoder_input_data[i,t,input_token_index[char]] = 1.0
    
    decoder_input_data[i,t+1 :,input_token_index[" "]] = 1.0 # encoding space at the end of encoder word for decoder input to reach to next word for encoding
    for t,char in enumerate(tar_word):
      decoder_input_data[i,t,target_token_index[char]] = 1.0
      if t>0:
        decoder_target_data[i,t-1,target_token_index[char]] = 1.0
    decoder_input_data[i,t+1 :,target_token_index[" "]] = 1.0
    decoder_target_data[i,t:,target_token_index[" "]] = 1.0
    
    
  
  save_dir = '/content/'
  #fold_var = 10
  #batch_size = 128
  #epochs = 25
  latent_dim = 256 # edit LSTM units
  num_samples = 14032
  data_path = 'min-ind_test.txt'
  input_texts = []
  target_texts = []
  #input_characters = set()
  #target_characters = set()
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  print('lines',lines[:len(lines)])
  print(len(lines))
  for line in lines[:num_samples]:
    if len(line.split(','))==2:
      input_text, target_text = line.split(",")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    
  '''
  '''
  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t-1, input_token_index[char]] = 1.0
  '''
  '''
  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t, input_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t, target_token_index[char]] = 1.0
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0
  '''
  
  df = pd.DataFrame()
  input_texts_ls = []
  decoded_sentences_ls = []
  target_texts_ls = []
  for seq_index in range(50):
      # Take one sequence (part of the training set)
      # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
      #print(input_seq)
    decoded_sentence= decode_sequence(input_seq)
    #input_texts_ls.append(np.array(input_encode)[seq_index].replace("\t", "").replace("\n", ""))
    #target_texts_ls.append(np.array(target_encode)[seq_index].replace("\t", "").replace("\n", ""))
    #decoded_sentences_ls.append(decoded_sentence.replace("\t", "").replace("\n", ""))
    #print(type(input_texts_ls))
    #print("-")
    print("Input sentence:", ''.join(input_encode[seq_index]))
    print("Decoded sentence:", ''.join(decoded_sentence))
    print("Target sentence:", ''.join(target_encode[seq_index]))
  #temp_list = [input_texts_ls, target_texts_ls, decoded_sentences_ls]
  
  #df = pd.DataFrame(temp_list).transpose()
  #df = df.rename(columns={0: 'input_text', 1: 'target_text', 2:'decode_sentence'})
  #print(df)
  #df_csv = df.to_csv('Input_decoded_sentences_5-fold-{}.csv'.format(i))
  ## Split min-ind3 text file into two different text files for encoding and decoding ##
  '''
  lines_1 = []
  lines_2 = []
  with open('min-ind_train.txt','r',encoding='utf-8') as min:
    for line in min.readlines():
      split = line.split(',')
      if split!=['\n']:
        lines_1.append(split[0])
        lines_2.append(split[1])
  with open('min-ind_input.txt','w',encoding='utf-8') as enc_txt:
    for line in lines_1:
      enc_txt.write(line+'\n')

  with open('min-ind_target.txt','w',encoding='utf-8') as tar_txt:
    for line in lines_2:
      tar_txt.write(line)

  import sentencepiece as spm
  model_inp = spm.SentencePieceTrainer.train("--input=min-ind_input.txt --model_prefix=input --vocab_size=30")
  sp_inp = spm.SentencePieceProcessor()
  sp_inp.load('input.model')
  #import sentencepiece as spm
  model_tar = spm.SentencePieceTrainer.train("--input=min-ind_target.txt --model_prefix=target --vocab_size=30")
  sp_tar = spm.SentencePieceProcessor()
  sp_tar.load('target.model')

  ### Prepare data ###
  input_encode = []
  target_encode = []
  with open('min-ind_input.txt','r',encoding='utf-8') as inp:
    for line in inp:
      input_encode.append(sp_inp.encode_as_pieces(line))
  with open('min-ind_target.txt','r',encoding='utf-8') as tar:
    for line in tar:
      target_encode.append(['\t']+sp_tar.encode_as_pieces(line)+['\n'])

  ## Create a dictionary for characters mapping ##
  input_subwords = np.unique(np.array([tk for tok in input_encode for tk in tok]))
  input_subwords = np.append(input_subwords," ")
  target_subwords = np.unique(np.array([tk for tok in target_encode for tk in tok]))
  target_subwords = np.append(target_subwords," ")
  input_token_index = dict([(char,i) for i,char in enumerate(sorted(list(input_subwords)))])
  target_token_index = dict([(char,i) for i,char in enumerate(sorted(list(target_subwords)))])

  num_encoder_tokens= len(input_subwords)
  num_decoder_tokens = len(target_subwords)

  max_encoder_seq_length = max([len(words) for words in input_encode])
  max_decoder_seq_length = max([len(words) for words in target_encode])

  encoder_input_data = np.zeros((len(input_encode),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
  decoder_input_data = np.zeros((len(input_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
  decoder_target_data = np.zeros((len(target_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

  for i,(enc_word,tar_word) in enumerate(zip(input_encode,target_encode)):
    for t,char in enumerate(enc_word):
      encoder_input_data[i,t,input_token_index[char]] = 1.0
    decoder_input_data[i,t+1 :,input_token_index[" "]] = 1.0 # encoding space at the end of encoder word for decoder input to reach to next word for encoding
    for t,char in enumerate(tar_word):
      decoder_input_data[i,t,target_token_index[char]] = 1.0
      if t>0:
        decoder_target_data[i,t-1,target_token_index[char]] = 1.0
    decoder_input_data[i,t+1 :,target_token_index[" "]] = 1.0
    decoder_target_data[i,t:,target_token_index[" "]] = 1.0



  
  data_path = 'min-ind_train.txt'
  input_texts = []
  target_texts = []
  input_characters = set()
  target_characters = set()
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    #print('lines',lines)
  print('lines',lines[:len(lines)])
  print(len(lines))
  for line in lines[:num_samples]:
    if len(line.split(','))==2:
      input_text, target_text = line.split(",")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
  input_characters.add(':')
  target_characters.add('-')
  input_characters = sorted(list(input_characters))
  target_characters = sorted(list(target_characters))
  num_encoder_tokens = len(input_characters)
  num_decoder_tokens = len(target_characters)
  max_encoder_seq_length = max([len(txt) for txt in input_texts])
  max_decoder_seq_length = max([len(txt) for txt in target_texts])
  '''
  '''
  input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
  target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

  encoder_input_data = np.zeros(
      (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
  )
  decoder_input_data = np.zeros(
      (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
  )
  decoder_target_data = np.zeros(
      (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
  )

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t, input_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t, target_token_index[char]] = 1.0
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0
  '''

K=1
Epoch 1/10
79/79 [==============================] - 39s 69ms/step - loss: 1.9247 - accuracy: 0.5696 - recall: 0.4195 - precision: 0.8768 - val_loss: 1.1186 - val_accuracy: 0.6857 - val_recall: 0.5948 - val_precision: 0.9835
Epoch 2/10
79/79 [==============================] - 3s 44ms/step - loss: 1.0922 - accuracy: 0.6910 - recall: 0.5996 - precision: 0.9772 - val_loss: 0.9541 - val_accuracy: 0.7287 - val_recall: 0.6331 - val_precision: 0.9932
Epoch 3/10
79/79 [==============================] - 4s 45ms/step - loss: 0.9374 - accuracy: 0.7284 - recall: 0.6388 - precision: 0.9889 - val_loss: 0.8665 - val_accuracy: 0.7397 - val_recall: 0.6556 - val_precision: 0.9831
Epoch 4/10
79/79 [==============================] - 3s 44ms/step - loss: 0.8710 - accuracy: 0.7372 - recall: 0.6544 - precision: 0.9791 - val_loss: 0.8215 - val_accuracy: 0.7505 - val_recall: 0.6660 - val_precision: 0.9801
Epoch 5/10
79/79 [==============================] - 3s 44ms/step - loss: 0.8312 - accuracy: 0.7450 - re

INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold1/assets


INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold1/assets


Input sentence: ▁hilang
Decoded sentence: ▁keramatan

Target sentence: 	▁hilang

Input sentence: ▁ekonomis
Decoded sentence: ▁keramatan

Target sentence: 	▁ekonomis

Input sentence: ▁d
Decoded sentence: ▁peratangan

Target sentence: 	▁d

Input sentence: ▁pamacik
Decoded sentence: ▁peramatan

Target sentence: 	▁pemegang

Input sentence: ▁sekunder
Decoded sentence: ▁keratangan

Target sentence: 	▁sekunder

Input sentence: ▁televisi
Decoded sentence: ▁pertendangan

Target sentence: 	▁televisi

Input sentence: ▁gletser
Decoded sentence: ▁peranggang

Target sentence: 	▁gletser

Input sentence: ▁adiah
Decoded sentence: ▁diperakat

Target sentence: 	▁hadiah

Input sentence: ▁fajar
Decoded sentence: ▁keramatan

Target sentence: 	▁fajar

Input sentence: ▁panangkok
Decoded sentence: ▁perakatan

Target sentence: 	▁penangkap

Input sentence: ▁kupu
Decoded sentence: ▁keratangan

Target sentence: 	▁kupu

Input sentence: ▁pangembangan
Decoded sentence: ▁peramatan

Target sentence: 	▁pengembangan

Inp

INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold2/assets


...

INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold9/assets


Input sentence: ▁hilang
Decoded sentence: ▁diperahasi

Target sentence: 	▁hilang

Input sentence: ▁ekonomis
Decoded sentence: ▁kenama

Target sentence: 	▁ekonomis

Input sentence: ▁d
Decoded sentence: ▁diserakan

Target sentence: 	▁d

Input sentence: ▁pamacik
Decoded sentence: ▁perambungan

Target sentence: 	▁pemegang

Input sentence: ▁sekunder
Decoded sentence: ▁kenukuan

Target sentence: 	▁sekunder

Input sentence: ▁televisi
Decoded sentence: ▁teralis

Target sentence: 	▁televisi

Input sentence: ▁gletser
Decoded sentence: ▁terata

Target sentence: 	▁gletser

Input sentence: ▁adiah
Decoded sentence: ▁disarahan

Target sentence: 	▁hadiah

Input sentence: ▁fajar
Decoded sentence: ▁berakaran

Target sentence: 	▁fajar

Input sentence: ▁panangkok
Decoded sentence: ▁pengatakan

Target sentence: 	▁penangkap

Input sentence: ▁kupu
Decoded sentence: ▁kerukusan

Target sentence: 	▁kupu

Input sentence: ▁pangembangan
Decoded sentence: ▁peranggangan

Target sentence: 	▁pengembangan

Input senten

INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold10/assets


INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold10/assets


Input sentence: ▁hilang
Decoded sentence: ▁diamasi

Target sentence: 	▁hilang

Input sentence: ▁ekonomis
Decoded sentence: ▁kolomonisisisi

Target sentence: 	▁ekonomis

Input sentence: ▁d
Decoded sentence: ▁disamatkan

Target sentence: 	▁d

Input sentence: ▁pamacik
Decoded sentence: ▁perambatan

Target sentence: 	▁pemegang

Input sentence: ▁sekunder
Decoded sentence: ▁sekata

Target sentence: 	▁sekunder

Input sentence: ▁televisi
Decoded sentence: ▁kelatisan

Target sentence: 	▁televisi

Input sentence: ▁gletser
Decoded sentence: ▁kerasatan

Target sentence: 	▁gletser

Input sentence: ▁adiah
Decoded sentence: ▁diamatkan

Target sentence: 	▁hadiah

Input sentence: ▁fajar
Decoded sentence: ▁berasah

Target sentence: 	▁fajar

Input sentence: ▁panangkok
Decoded sentence: ▁perakatan

Target sentence: 	▁penangkap

Input sentence: ▁kupu
Decoded sentence: ▁kurus

Target sentence: 	▁kupu

Input sentence: ▁pangembangan
Decoded sentence: ▁peranggatan

Target sentence: 	▁pengembangan

Input senten